In [159]:
import random

import pandas as pd
import numpy as np
from scipy import stats

# from oauth2client.service_account import ServiceAccountCredentials 
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials 

In [5]:
CREDENTIALS_PATH_GOOGLE = 'google-credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET = '1b75J-QTGrujSgF9r0_JPOKkcXAwzFVwpETOAyVBw8ak'

In [6]:
# Load service account credentials.
__credentials = Credentials.from_service_account_file(CREDENTIALS_PATH_GOOGLE, scopes=SCOPES)

# Creates Google Sheets API (v4/latest) service.
service = build('sheets', 'v4', credentials=__credentials)

In [22]:
# Gets values from Ach! Musik: Notations sheet.
values = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET, range='Notations').execute()['values']
headers = values.pop(0)

In [31]:
# Format data as pd.DataFrame
data = pd.DataFrame(values, columns=headers)
data

ntQ  \
genre sub_genre  artist         album           song                                  
Pop   Disco      Abba           Abba            I do, I do, I do, I do, I do      8   
                                                Mamma Mia                       8,5   
                                                SOS                               9   
                                Arrival         Knowing me, knowing you         7,5   
                                Super Trouper   Lay all your love on me           8   
...                                                                             ...   
Rock  Hard Rock  ZZ Top         Eliminator      Sharp Dressed Man                 9   
                                                Thug                           8,25   
                                                TV Dinners                     7,75   
      Surf Vocal The Beach Boys All Summer Long I Get Around                    8,5   
                 Marvin Gaye                    Ain't No Mountain High Enough  None   

                                                                                ntG  \
genre sub_genre  artist         album           song                                  
Pop   Disco      Abba           Abba            I do, I do, I do, I do, I do    8,5   
                                                Mamma Mia                      8,75   
                                                SOS                               9   
                                Arrival         Knowing me, knowing you         8,5   
                                Super Trouper   Lay all your love on me         8,5   
...                                                                             ...   
Rock  Hard Rock  ZZ Top         Eliminator      Sharp Dressed Man                     
                                                Thug                                  
                                                TV Dinners                            
      Surf Vocal The Beach Boys All Summer Long I Get Around                   None   
                 Marvin Gaye                    Ain't No Mountain High Enough  None   

                                                                                ntV  \
genre sub_genre  artist         album           song                                  
Pop   Disco      Abba           Abba            I do, I do, I do, I do, I do      5   
                                                Mamma Mia                       6,5   
                                                SOS                               7   
                                Arrival         Knowing me, knowing you         5,5   
                                Super Trouper   Lay all your love on me           4   
...                                                                             ...   
Rock  Hard Rock  ZZ Top         Eliminator      Sharp Dressed Man               8,5   
                                                Thug                           8,75   
                                                TV Dinners                        7   
      Surf Vocal The Beach Boys All Summer Long I Get Around                   None   
                 Marvin Gaye                    Ain't No Mountain High Enough  None   

                                                                                ntR  \
genre sub_genre  artist         album           song                                  
Pop   Disco      Abba           Abba            I do, I do, I do, I do, I do      8   
                                                Mamma Mia                       8,3   
                                                SOS                             8,4   
                                Arrival         Knowing me, knowing you         6,5   
                                Super Trouper   Lay all your love on me         8,5   
...                                                                        

In [9]:
# Saving as csv for later use
data.to_csv("../data/achmusik.csv", index=True, decimal=",")

In [183]:
data = pd.read_csv("../data/achmusik.csv")

In [184]:
# Getting the decimals right -- commas to points and no more Nones
data = data.set_index(["genre", "sub_genre", "artist", "album", "song"])
data.fillna(value="", inplace=True)

for i in range(data.columns.size):
    data[data.columns[i]] = data[data.columns[i]].str.replace(",", ".")
    data[data.columns[i]] = pd.to_numeric(data[data.columns[i]], errors='coerce')

In [185]:
kept_people = ["Qu", "Gr", "Vi", "Ro"]
default_grade = 5

# Keeping only present people at the hypothetical party!
data = data.filter(kept_people)

# Hard to do this shit inplace -- if no grades at all, give it a chance to play with default grade
data = data.dropna(how="all").append(data[data.isnull().all(axis=1)].fillna(default_grade))
data

Qu  \
genre sub_genre   artist            album           song                                
Pop   Disco       Abba              Abba            I do, I do, I do, I do, I do  8.0   
                                                    Mamma Mia                     8.5   
                                                    SOS                           9.0   
                                    Arrival         Knowing me, knowing you       7.5   
                                    Super Trouper   Lay all your love on me       8.0   
...                                                                               ...   
Metal Symphonique Within Temptation The Unforgiving Murder                        5.0   
                                                    Shot in the Dark              5.0   
                                                    Sinead                        5.0   
                                                    Stairway to the Skies         5.0   
                                                    Where is the Edge             5.0   

                                                                                    Gr  \
genre sub_genre   artist            album           song                                 
Pop   Disco       Abba              Abba            I do, I do, I do, I do, I do  8.50   
                                                    Mamma Mia                     8.75   
                                                    SOS                           9.00   
                                    Arrival         Knowing me, knowing you       8.50   
                                    Super Trouper   Lay all your love on me       8.50   
...                                                                                ...   
Metal Symphonique Within Temptation The Unforgiving Murder                        5.00   
                                                    Shot in the Dark              5.00   
                                                    Sinead                        5.00   
                                                    Stairway to the Skies         5.00   
                                                    Where is the Edge             5.00   

                                                                                   Vi  \
genre sub_genre   artist            album           song                                
Pop   Disco       Abba              Abba            I do, I do, I do, I do, I do  5.0   
                                                    Mamma Mia                     6.5   
                                                    SOS                           7.0   
                                    Arrival         Knowing me, knowing you       5.5   
                                    Super Trouper   Lay all your love on me       4.0   
...                                                                               ...   
Metal Symphonique Within Temptation The Unforgiving Murder                        5.0   
                                                    Shot in the Dark              5.0   
                                                    Sinead                        5.0   
                                                    Stairway to the Skies         5.0   
                                                    Where is the Edge             5.0   

                                                                                   Ro  
genre sub_genre   artist            album           song                               
Pop   Disco       Abba              Abba            I do, I do, I do, I do, I do  8.0  
                                                    Mamma Mia                     8.3  
                                                    SOS                           8.4  
                                    Arrival         Knowing me, knowing you       6.5  
                                    Super Trouper   Lay all your lov

## Score voting

In [186]:
COUNT_FACTOR = .1
COUNT_INHIB = len(kept_people) // 2
MIN_SCORE = 5
PLAYLIST_SIZE = 300  # Can be an int or a fraction 0 < q <= 1

# To avoid having to hard code the amount of columns for cases where the next cell is re-ran, we initialize columns
data["mean"] = 0
data["count"] = 0
data["score"] = 0
data["rank"] = 0

In [187]:
# Mean of all notes for each track
data["mean"] = data[data.columns[:-4]].mean(axis=1)

# Amount of notes for each track
data["count"] = data.count(axis=1) - 4

# Helping songs graded by more people in the group
data["score"] = data["mean"] + (COUNT_FACTOR * (data["count"] - COUNT_INHIB))

# Truncating to keep only the acceptable songs
data = data[data["score"] > MIN_SCORE]

data = data.sort_values("score", ascending=False)
data["rank"] = data["score"].rank(method="min")
data

Qu  \
genre   sub_genre   artist            album                        song                                  
Rock    NaN         Dire Straits      Dire Straits                 Sultans Of Swing              10.00   
        Acoustic    Eric Clapton      Unplugged                    Layla                         10.00   
        NaN         The Beatles       Abbey Road                   Here Comes The Sun            10.00   
        Progressive Pink Floyd        Animals                      Pigs (Three Different Ones)    9.75   
        NaN         The Beatles       The Beatles                  While My Guitar Gently Weeps   9.75   
...                                                                                                ...   
Metal   Symphonique Within Temptation The Unforgiving              Where is the Edge              5.00   
Pop     Indie       Charlotte Cardin  Main Girl                    Main Girl                      5.25   
Hip-Hop Punk        Stupeflip         The Hypnoflip Invasion       Dangereux !!                   5.25   
Reggae  Ska-Punk    The Skints        Swimming Lessons             Gets on Top                     NaN   
Pop     Dance       Madonna           Confessions on a Dance Floor Hung Up                        3.00   

                                                                                                   Gr  \
genre   sub_genre   artist            album                        song                                 
Rock    NaN         Dire Straits      Dire Straits                 Sultans Of Swing               NaN   
        Acoustic    Eric Clapton      Unplugged                    Layla                          NaN   
        NaN         The Beatles       Abbey Road                   Here Comes The Sun            9.25   
        Progressive Pink Floyd        Animals                      Pigs (Three Different Ones)    NaN   
        NaN         The Beatles       The Beatles                  While My Guitar Gently Weeps  9.50   
...                                                                                               ...   
Metal   Symphonique Within Temptation The Unforgiving              Where is the Edge             5.00   
Pop     Indie       Charlotte Cardin  Main Girl                    Main Girl                      NaN   
Hip-Hop Punk        Stupeflip         The Hypnoflip Invasion       Dangereux !!                   NaN   
Reggae  Ska-Punk    The Skints        Swimming Lessons             Gets on Top                    NaN   
Pop     Dance       Madonna           Confessions on a Dance Floor Hung Up                        NaN   

                                                                                                   Vi  \
genre   sub_genre   artist            album                        song                                 
Rock    NaN         Dire Straits      Dire Straits                 Sultans Of Swing               NaN   
        Acoustic    Eric Clapton      Unplugged                    Layla                         9.75   
        NaN         The Beatles       Abbey Road                   Here Comes The Sun            9.50   
        Progressive Pink Floyd        Animals                      Pigs (Three Different Ones)    NaN   
        NaN         The Beatles       The Beatles                  While My Guitar Gently Weeps   NaN   
...                                                                                               ...   
Metal   Symphonique Within Temptation The Unforgiving              Where is the Edge             5.00   
Pop     Indie       Charlotte Cardin  Main Girl                    Main Girl                      NaN   
Hip-Hop Punk        Stupeflip         The Hypnoflip Invasion       Dangereux !!                  5.00   
Reggae  Ska-Punk    The Skints        Swimming Lessons             Gets on Top                    NaN   
Pop     Dance       Madonna           Confessions on a Dance Floor Hung Up                     

In [135]:
if PLAYLIST_SIZE < 1:
    playlist = data.sample(frac=PLAYLIST_SIZE, weights="rank")
else:
    playlist = data.sample(n=PLAYLIST_SIZE, weights="rank")

playlist

ntQ  \
genre   sub_genre     artist             album                        song                                       
Hip Hop NaN           MC Hammer          Please Hammer Don't Hurt 'Em U Can't Touch This                  7.75   
Metal   Melodic Death Amon Amarth        Berserker                    Crack the Sky                       8.50   
        Power Metal   Powerwolf          The Sacrament of Sin         Fist by Fist (Sacralize or Strike)   NaN   
        Thrash Metal  Metallica          ...And Justice for All       Blackened                            NaN   
        Thrash        Megadeth           Rust in Peace                Hangar 18                           9.00   
...                                                                                                        ...   
Rock    Hard Rock     ZZ Top             Eliminator                   Thug                                8.25   
Metal   Doom/Gothic   Paradise Lost      Draconian Times              Shadowkings                         7.00   
Jazz    OST           Kristofer Maddigan Cuphead                      Inkwell Hell                        8.50   
Metal   Progressive   Tool               Lateralus                    Parabola                             NaN   
        Death         Gojira             The Link                     Dawn                                7.50   

                                                                                                          ntG  \
genre   sub_genre     artist             album                        song                                      
Hip Hop NaN           MC Hammer          Please Hammer Don't Hurt 'Em U Can't Touch This                  NaN   
Metal   Melodic Death Amon Amarth        Berserker                    Crack the Sky                       8.5   
        Power Metal   Powerwolf          The Sacrament of Sin         Fist by Fist (Sacralize or Strike)  NaN   
        Thrash Metal  Metallica          ...And Justice for All       Blackened                           8.5   
        Thrash        Megadeth           Rust in Peace                Hangar 18                           9.0   
...                                                                                                       ...   
Rock    Hard Rock     ZZ Top             Eliminator                   Thug                                NaN   
Metal   Doom/Gothic   Paradise Lost      Draconian Times              Shadowkings                         7.5   
Jazz    OST           Kristofer Maddigan Cuphead                      Inkwell Hell                        NaN   
Metal   Progressive   Tool               Lateralus                    Parabola                            9.0   
        Death         Gojira             The Link                     Dawn                                6.5   

                                                                                                           ntV  \
genre   sub_genre     artist             album                        song                                       
Hip Hop NaN           MC Hammer          Please Hammer Don't Hurt 'Em U Can't Touch This                   NaN   
Metal   Melodic Death Amon Amarth        Berserker                    Crack the Sky                       6.50   
        Power Metal   Powerwolf          The Sacrament of Sin         Fist by Fist (Sacralize or Strike)   NaN   
        Thrash Metal  Metallica          ...And Justice for All       Blackened                            NaN   
        Thrash        Megadeth           Rust in Peace                Hangar 18                            NaN   
...                                                                                                        ...   
Rock    Hard Rock     ZZ Top             Eliminator                   Thug                                8.75   
Metal   Doom/Gothic   Paradise Lost      Draconian Times              Shadowkings                          NaN   
Jazz    OST   

In [177]:
ADJACENT_GENRES = {"Metal": ["Rock", "Metal", "Grunge"], 
                   "Rock": ["Metal", "Rock", "Punk", "Grunge", "Pop", "Blues"], 
                   "Jazz": ["Classique", "Jazz", "Acoustic", "Blues"],
                   "Classique": ["Jazz", "Classique", "Acoustic"]
                   "Electro": ["Dance", "Pop", "Electro", "House"],
                   "Dance": ["Dance", "Electro", "Pop", "Disco", "House", "Funk"],
                   "Funk": ["Funk", "Disco", "House", "Pop"],
                   "Pop": ["Funk", "Disco", "House", "Electro", "Dance", "Rock"]}

# Listing genres to prepare distance matrix
genres_labels = pd.DataFrame([genre for genre in playlist.reset_index()["genre"].unique()], columns=["genre"])
# genres_labels["index"] = np.arange(0, len(genres), 1)
genres_labels

genres_labels

,genre
0,Hip Hop
1,Metal
2,Rock
3,Funk
4,Jazz
5,Electro
6,Disco
7,Chanson
8,Classique
9,Pop


In [163]:
# Rearranging playlist to avoid sudden genre changes
genres = [playlist for _, playlist in playlist.groupby("genre")]
random.shuffle(genres)

pd.concat(genres)

ntQ  \
genre   sub_genre     artist      album                  song                                       
Metal   Melodic Death Amon Amarth Berserker              Crack the Sky                       8.50   
        Power Metal   Powerwolf   The Sacrament of Sin   Fist by Fist (Sacralize or Strike)   NaN   
        Thrash Metal  Metallica   ...And Justice for All Blackened                            NaN   
        Thrash        Megadeth    Rust in Peace          Hangar 18                           9.00   
        Progressive   Leprous     The Congregation       The Flood                           9.00   
...                                                                                           ...   
Electro Alt Rock      Gorillaz    Demon Days             Kids With Guns                      9.75   
        Pop           Parcels     Live Vol. 1            Redline                             8.00   
        Alt Rock      Gorillaz    Plastic Beach          On Melancholy Hill                  8.50   
        Pop           Parcels     Live Vol. 1            Overnight                           8.25   
                                                         Lightenup                           8.50   

                                                                                             ntG  \
genre   sub_genre     artist      album                  song                                      
Metal   Melodic Death Amon Amarth Berserker              Crack the Sky                       8.5   
        Power Metal   Powerwolf   The Sacrament of Sin   Fist by Fist (Sacralize or Strike)  NaN   
        Thrash Metal  Metallica   ...And Justice for All Blackened                           8.5   
        Thrash        Megadeth    Rust in Peace          Hangar 18                           9.0   
        Progressive   Leprous     The Congregation       The Flood                           8.0   
...                                                                                          ...   
Electro Alt Rock      Gorillaz    Demon Days             Kids With Guns                      NaN   
        Pop           Parcels     Live Vol. 1            Redline                             NaN   
        Alt Rock      Gorillaz    Plastic Beach          On Melancholy Hill                  NaN   
        Pop           Parcels     Live Vol. 1            Overnight                           NaN   
                                                         Lightenup                           NaN   

                                                                                              ntV  \
genre   sub_genre     artist      album                  song                                       
Metal   Melodic Death Amon Amarth Berserker              Crack the Sky                       6.50   
        Power Metal   Powerwolf   The Sacrament of Sin   Fist by Fist (Sacralize or Strike)   NaN   
        Thrash Metal  Metallica   ...And Justice for All Blackened                            NaN   
        Thrash        Megadeth    Rust in Peace          Hangar 18                            NaN   
        Progressive   Leprous     The Congregation       The Flood                            NaN   
...                                                                                           ...   
Electro Alt Rock      Gorillaz    Demon Days             Kids With Guns                      9.25   
        Pop           Parcels     Live Vol. 1            Redline                             8.75   
        Alt Rock      Gorillaz    Plastic Beach          On Melancholy Hill                  8.50   
        Pop           Parcels     Live Vol. 1            Overnight                           9.00   
                                                         Lightenup                           9.00   

                                                                                             ntR  \
genre   sub_genre     artist      album                  song    

In [78]:
top_artists = data.groupby("artist").filter(lambda x: len(x) > 4).groupby("artist").mean()["mean"].sort_values(ascending=False).tail(10)
top_artists

artist
Nightwish            6.856818
Leprous              6.773551
Paradise Lost        6.763889
Stand High Patrol    6.655556
The Skints           6.631250
Herbie Hancock       6.621429
Happy End            6.543403
Within Temptation    6.177604
Neurosis             5.879167
The Pretenders       5.590909
Name: mean, dtype: float64

## Transforming notes into ranks

In [102]:
# Default pandas ranking
for i in range (5, data.columns.size):
    data[data.columns[i]] = data[data.columns[i]].rank()
data

,genre,sub_genre,artist,album,song,ntQ,ntG,ntV,ntR,ntS,ntGl,ntRx,ntC,ntL
0,Pop,Disco,Abba,Abba,"I do, I do, I do, I do, I do",701.5,347.0,43.5,107.0,NaN,NaN,NaN,NaN,3.0
1,Pop,Disco,Abba,Abba,Mamma Mia,980.5,434.0,162.0,131.0,NaN,8.5,NaN,NaN,NaN
2,Pop,Disco,Abba,Abba,SOS,1170.0,503.5,242.5,138.5,NaN,11.0,NaN,NaN,NaN
3,Pop,Disco,Abba,Arrival,"Knowing me, knowing you",377.5,347.0,58.5,26.0,NaN,NaN,NaN,NaN,NaN
4,Pop,Disco,Abba,Super Trouper,Lay all your love on me,701.5,347.0,16.0,143.5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,Rock,Hard Rock,ZZ Top,Eliminator,If I Could Only Flag Her Down,377.5,NaN,242.5,NaN,NaN,NaN,NaN,NaN,NaN
1486,Rock,Hard Rock,ZZ Top,Eliminator,Legs,1219.5,NaN,461.5,150.5,NaN,NaN,NaN,10.0,NaN
1487,Rock,Hard Rock,ZZ Top,Eliminator,Sharp Dressed Man,1170.0,NaN,582.5,179.0,NaN,NaN,NaN,NaN,NaN
1488,Rock,Hard Rock,ZZ Top,Eliminator,Thug,846.0,NaN,641.5,NaN,NaN,NaN,NaN,NaN,NaN
